In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score

import numpy as np

from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

from scipy.sparse import hstack

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LEGION\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LEGION\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LEGION\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Load the datasets
train_data = pd.read_csv('train.csv')
val_data = pd.read_csv('val.csv')
test_data = pd.read_csv('test.csv')

lengthTestData = len(test_data)

In [3]:
# Combine text data for preprocessing
text = pd.concat([train_data['text'], val_data['text']], ignore_index=True)
text_test = test_data['text']

In [4]:
linguistic_features = pd.concat([train_data.iloc[:,-9:], val_data.iloc[:,-9:]], ignore_index=True)
test_linguistic_features = test_data.iloc[:,-9:]

In [5]:
linguistic_features

,Words_per_Sentence,Percentage_Questions,Percentage_First_Person_Singular,Percentage_Second_Person,Percentage_Third_Person,Percentage_Negation,Percentage_Exclusive,Percentage_Causation,Percentage_Sense
0,28.777778,0.000000,0.000000,0.000000,1.930502,0.000000,0.000000,2.702703,5.405405
1,32.900000,0.000000,0.000000,0.000000,2.735562,0.607903,1.519757,1.519757,4.863222
2,30.416667,0.000000,0.000000,0.000000,2.191781,0.547945,0.821918,1.095890,6.027397
3,21.254545,3.636364,0.085543,0.000000,4.790419,1.197605,1.710864,2.138580,3.079555
4,24.444444,0.000000,0.000000,0.090909,3.181818,0.545455,1.636364,2.181818,6.181818
...,...,...,...,...,...,...,...,...,...
54714,56.166667,16.666667,0.296736,0.000000,4.154303,1.186944,1.186944,0.593472,5.341246
54715,22.900000,0.000000,0.000000,0.000000,2.183406,0.436681,0.873362,0.000000,5.240175
54716,18.095238,14.285714,1.842105,3.684211,5.263158,1.052632,2.105263,1.315789,5.789474
54717,16.611354,10.043668,1.156677,2.576236,2.970557,1.445846,2.602524,1.393270,2.628812


In [6]:
test_linguistic_features

,Words_per_Sentence,Percentage_Questions,Percentage_First_Person_Singular,Percentage_Second_Person,Percentage_Third_Person,Percentage_Negation,Percentage_Exclusive,Percentage_Causation,Percentage_Sense
0,25.066667,0.000000,0.000000,0.265957,2.393617,0.000000,1.595745,2.393617,7.446809
1,23.333333,0.000000,0.000000,0.000000,2.857143,2.857143,2.857143,2.142857,2.857143
2,24.333333,0.000000,0.000000,0.000000,1.369863,1.369863,1.369863,0.000000,8.219178
3,27.466667,0.000000,0.404531,0.242718,3.559871,0.485437,0.566343,1.779935,3.478964
4,40.666667,0.000000,0.000000,0.000000,3.278689,0.000000,0.819672,3.278689,6.557377
...,...,...,...,...,...,...,...,...,...
6076,26.708333,8.333333,0.468019,0.624025,4.680187,0.936037,2.184087,1.560062,1.872075
6077,32.700000,0.000000,0.000000,0.000000,1.834862,0.000000,0.305810,0.000000,4.587156
6078,25.600000,0.000000,0.000000,0.000000,4.427083,0.520833,0.781250,3.125000,2.083333
6079,63.000000,0.000000,0.000000,0.000000,5.555556,0.000000,2.777778,0.793651,2.380952


In [7]:
# Define stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [8]:
# Function to preprocess text
def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenization and lowercase
    filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]  # Remove stopwords and lemmatize
    return ' '.join(filtered_tokens)

In [9]:
text_preprocessed = text.apply(preprocess_text)

In [10]:
print(text_preprocessed)

0        beirut ( reuters ) - iran military chief met s...
1        hanoi ( reuters ) - top u.s. envoy began two-d...
2        ( reuters ) - four u.s. senator asked senate j...
3        first read morning briefing meet press nbc pol...
4        cairo ( reuters ) - six month egypt election ,...
                               ...                        
54714    lack oversight prof donald trump totally unfit...
54715    tucker carlson responded espn anchor calling p...
54716    getting something nothing rage president profe...
54717    black emanuelle fixed 1976. attila speaking eu...
54718    chaos broke legal american illegal alien clash...
Name: text, Length: 54719, dtype: object


In [11]:
print(text)

0        beirut (reuters) - iran s military chief met w...
1        hanoi (reuters) - a top u.s. envoy began a two...
2        (reuters) - four u.s. senators have asked the ...
3        first read is a morning briefing from meet the...
4        cairo (reuters) - six months before egypt s el...
                               ...                        
54714    this lack of oversight proves that donald trum...
54715    tucker carlson responded to an espn anchor cal...
54716    because getting something for nothing is all t...
54717    black emanuelle fixed all that in 1976. attila...
54718    chaos broke out after legal americans and ille...
Name: text, Length: 54719, dtype: object


In [12]:
# Initialize TF-IDF vectorizer without specifying max_features
tfidf_vectorizer = TfidfVectorizer()

In [13]:
# Fit and transform the preprocessed text data
tfidf_matrix = tfidf_vectorizer.fit_transform(text_preprocessed)

In [14]:
# Get the number of unique tokens
num_unique_tokens = len(tfidf_vectorizer.get_feature_names_out())
print(num_unique_tokens)

169079


In [15]:
# Re-initialize TF-IDF vectorizer with the determined max_features
tfidf_vectorizer = TfidfVectorizer(max_features=num_unique_tokens)

In [16]:
#Fit and transform the text data again with the updated max_features
tfidf_matrix = tfidf_vectorizer.fit_transform(text_preprocessed)

# Convert the TF-IDF matrix to a CSR (Compressed Sparse Row) matrix for efficient row-wise operations
csr_tfidf_matrix = csr_matrix(tfidf_matrix)

# Find the row index with the maximum number of filled values
max_features_row_index = csr_tfidf_matrix.getnnz(axis=1).argmax()

# Get the number of features in the document with the most filled values
max_features = csr_tfidf_matrix[max_features_row_index].count_nonzero()

svd = TruncatedSVD(n_components=int(max_features*0.3))
tfidf_matrix = svd.fit_transform(tfidf_matrix)

In [17]:
#dense_tfidf_matrix = tfidf_matrix[:len(train_data)]
#dense_val_tfidf_matrix = tfidf_matrix[len(train_data):len(train_data) + len(val_data)]

# Merging the Validation and Training Data into one for a larger training dataset.
#dense_tfidf_matrix = tfidf_matrix[:len(train_data) + len(val_data)]

In [18]:
# Concatenate linguistic features and TF-IDF matrix horizontally
dense_tfidf_with_linguistic = hstack([tfidf_matrix, csr_matrix(linguistic_features)])

In [19]:
# Convert them into Arrays
train_labels = train_data['label'].values
val_labels = val_data['label'].values

dense_labels = np.concatenate((train_data['label'].values, val_data['label'].values), axis=0)

In [20]:
# Create and train the MLP model
mlp_model = MLPClassifier(hidden_layer_sizes=(600, 10), max_iter=300, activation='relu', solver='adam', learning_rate='adaptive', verbose=True)

In [21]:
#CROSS FOLD VALIDATION, DELETE IF NECESSARY

# Initialize KFold with the desired number of folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(mlp_model, dense_tfidf_with_linguistic, dense_labels, cv=kf, scoring='accuracy')

# Print the cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Average CV accuracy:", cv_scores.mean())

Iteration 1, loss = 0.37072631
Iteration 2, loss = 0.14625432
Iteration 3, loss = 0.12174586
Iteration 4, loss = 0.11074755
Iteration 5, loss = 0.11026996
Iteration 6, loss = 0.10514726
Iteration 7, loss = 0.10468376
Iteration 8, loss = 0.09158401
Iteration 9, loss = 0.08801302
Iteration 10, loss = 0.08264857
Iteration 11, loss = 0.07460030
Iteration 12, loss = 0.07454546
Iteration 13, loss = 0.06756463
Iteration 14, loss = 0.05969429
Iteration 15, loss = 0.05578566
Iteration 16, loss = 0.05426848
Iteration 17, loss = 0.04497476
Iteration 18, loss = 0.04038199
Iteration 19, loss = 0.03768006
Iteration 20, loss = 0.03174275
Iteration 21, loss = 0.03395739
Iteration 22, loss = 0.02494772
Iteration 23, loss = 0.01996430
Iteration 24, loss = 0.01838851
Iteration 25, loss = 0.01595186
Iteration 26, loss = 0.01255287
Iteration 27, loss = 0.01219638
Iteration 28, loss = 0.01505597
Iteration 29, loss = 0.01269621
Iteration 30, loss = 0.01363711
Iteration 31, loss = 0.01835012
Iteration 32, los

In [22]:
mlp_model.fit(dense_tfidf_with_linguistic, dense_labels)

Iteration 1, loss = 0.35462215
Iteration 2, loss = 0.14054913
Iteration 3, loss = 0.12020714
Iteration 4, loss = 0.11031493
Iteration 5, loss = 0.10679876
Iteration 6, loss = 0.10506259
Iteration 7, loss = 0.09704173
Iteration 8, loss = 0.09375698
Iteration 9, loss = 0.08575828
Iteration 10, loss = 0.08181649
Iteration 11, loss = 0.07909204
Iteration 12, loss = 0.07542844
Iteration 13, loss = 0.06773307
Iteration 14, loss = 0.06388863
Iteration 15, loss = 0.05607777
Iteration 16, loss = 0.05136024
Iteration 17, loss = 0.05091632
Iteration 18, loss = 0.04646780
Iteration 19, loss = 0.04145968
Iteration 20, loss = 0.03412930
Iteration 21, loss = 0.03123033
Iteration 22, loss = 0.02836675
Iteration 23, loss = 0.03000259
Iteration 24, loss = 0.02396667
Iteration 25, loss = 0.01882743
Iteration 26, loss = 0.01699410
Iteration 27, loss = 0.01711680
Iteration 28, loss = 0.01429609
Iteration 29, loss = 0.01167754
Iteration 30, loss = 0.01258848
Iteration 31, loss = 0.01138063
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(600, 10), learning_rate='adaptive',
              max_iter=300, verbose=True)

In [23]:
text_test_preprocessed = text_test.apply(preprocess_text)
test_tfidf_matrix = tfidf_vectorizer.transform(text_test_preprocessed)
dense_test_tfidf_matrix = svd.transform(test_tfidf_matrix)
dense_test_tfidf_with_linguistic = hstack([dense_test_tfidf_matrix, csr_matrix(test_linguistic_features)])
test_labels = test_data['label'].values

test_predictions = mlp_model.predict(dense_test_tfidf_with_linguistic)
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f'Test accuracy: {test_accuracy}')

Test accuracy: 0.9541193882585101


In [24]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(test_labels, test_predictions)
print(report)   

              precision    recall  f1-score   support

           0       0.95      0.97      0.96      3420
           1       0.96      0.93      0.95      2661

    accuracy                           0.95      6081
   macro avg       0.96      0.95      0.95      6081
weighted avg       0.95      0.95      0.95      6081



In [25]:
from joblib import dump
dump(mlp_model, 'mlp_model_tfidf.joblib')

['mlp_model_tfidf.joblib']

In [26]:
print(lengthTestData)

6081


In [27]:
test_accuracy*lengthTestData

5802.0